In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(1, '../SyMBac/') # Not needed if you installed SyMBac using pip
sys.path.insert(1, '../SyMBac/') # Not needed if you installed SyMBac using pip

import ray
from SyMBac.drawing import raster_cell, OPL_to_FL
from SyMBac.PSF import PSF_generator
from SyMBac.renderer import convolve_rescale
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
from PIL import Image
from skimage.util import img_as_uint
import os
from joblib import Parallel, delayed
from glob import glob
from SyMBac.colony_renderer import ColonyRenderer
import noise
from skimage.filters import threshold_otsu
from skimage.transform import rescale, resize, downscale_local_mean
from scipy.signal import find_peaks
import pandas as pd

In [3]:
def perlin_generator(shape, resize_amount, scale = 5, octaves = 10, persistence = 1.9, lacunarity = 1.8):

        y, x = np.round(shape[0] / resize_amount).astype(int), np.round(shape[1] / resize_amount).astype(int)

        world = np.zeros((x, y))

        # make coordinate grid on [0,1]^2
        x_idx = np.linspace(0, 1, y)
        y_idx = np.linspace(0, 1, x)
        world_x, world_y = np.meshgrid(x_idx, y_idx)

        # apply perlin noise, instead of np.vectorize, consider using itertools.starmap()
        world = np.vectorize(noise.pnoise2)(world_x / scale,
                                            world_y / scale,
                                            octaves=octaves,
                                            persistence=persistence,
                                            lacunarity=lacunarity)

        # here was the error: one needs to normalize the image first. Could be done without copying the array, though
        img = np.floor((world + .5) * 255).astype(np.uint8)  # <- Normalize world first
        return img


In [4]:
from numba import njit

In [5]:
def raster_cell(length, width, separation = 0, additional_width = 0, pinching=True):
    """
    Produces a rasterised image of a cell with the intensiity of each pixel corresponding to the optical path length
    (thickness) of the cell at that point.

    :param int length: Cell length in pixels
    :param int width: Cell width in pixels
    :param int separation: An int between (0, `width`) controlling how much pinching is happening.
    :param bool pinching: Controls whether pinching is happening

    Returns
    -------

    cell : np.array
       A numpy array which contains an OPL image of the cell. Can be converted to a mask by just taking ``cell > 0``.

    """

    L = int(np.rint(length))
    W = int(np.rint(width))
    new_cell = np.zeros((L, W))
    R = (W - 1) / 2

    x_cyl = np.arange(0, 2 * R + 1, 1)
    I_cyl = np.sqrt(R ** 2 - (x_cyl - R) ** 2)
    L_cyl = L - W
    new_cell[int(W / 2):-int(W / 2), :] = I_cyl

    x_sphere = np.arange(0, int(W / 2), 1)
    sphere_Rs = np.sqrt((R) ** 2 - (x_sphere - R) ** 2)
    sphere_Rs = np.rint(sphere_Rs).astype(int)

    for c in range(len(sphere_Rs)):
        R_ = sphere_Rs[c]
        x_cyl = np.arange(0, R_, 1)
        I_cyl = np.sqrt(R_ ** 2 - (x_cyl - R_) ** 2)
        new_cell[c, int(W / 2) - sphere_Rs[c]:int(W / 2) + sphere_Rs[c]] = np.concatenate((I_cyl, I_cyl[::-1]))
        new_cell[L - c - 1, int(W / 2) - sphere_Rs[c]:int(W / 2) + sphere_Rs[c]] = np.concatenate((I_cyl, I_cyl[::-1]))

    if separation > 2 and pinching:
        S = int(np.rint(separation))
        new_cell[int((L - S) / 2) + 1:-int((L - S) / 2) - 1, :] = 0
        for c in range(int((S+1) / 2)):
            R__ = sphere_Rs[-c - 1]
            x_cyl_ = np.arange(0, R__, 1)
            I_cyl_ = np.sqrt(R__ ** 2 - (x_cyl_ - R__) ** 2)
            new_cell[int((L-S) / 2) + c + 1, int(W / 2) - R__:int(W / 2) + R__] = np.concatenate((I_cyl_, I_cyl_[::-1]))
            new_cell[-int((L-S) / 2) - c - 1, int(W / 2) - R__:int(W / 2) + R__] = np.concatenate((I_cyl_, I_cyl_[::-1]))
    new_cell = new_cell.astype(int)
            
    
    if additional_width>=1:
        additional_width = int(additional_width)
        wide_cell = np.zeros((new_cell.shape[0], int(new_cell.shape[1] + additional_width)))
        wide_cell[:,:new_cell.shape[1]//2] = new_cell[:,:new_cell.shape[1]//2]
        wide_cell[:,new_cell.shape[1]//2 + additional_width:] = new_cell[:,new_cell.shape[1]//2:]
        wide_cell[:,new_cell.shape[1]//2:new_cell.shape[1]//2 + additional_width] = np.repeat(new_cell[:,new_cell.shape[1]//2].reshape(-1,1), additional_width, axis=1)
        return wide_cell
    
    if additional_width<=1:
        new_cell[np.where(new_cell)] += int(abs(additional_width))
    
    return new_cell

In [6]:
@njit
def generate_deviation_from_CL(centreline, thickness):
    return np.arange(thickness) + centreline - int(np.ceil(thickness ))

@njit
def gen_3D_coords_from_2D(test_cells, centreline, thickness):
    return np.where(test_cells == thickness) + (generate_deviation_from_CL(centreline, thickness),)

@njit
def convert_to_3D_numba(cell):
    expanded_scene = cell
    volume_shape = expanded_scene.shape[0:] + (int(expanded_scene.max()*2),)
    test_cells = rounder(expanded_scene)
    centreline = int(expanded_scene.max() )
    cells_3D = np.zeros(volume_shape,dtype = np.ubyte)
    for t in range(int(expanded_scene.max() *2 )):
        test_coords = gen_3D_coords_from_2D(test_cells, centreline, t)
        for x, y in zip(test_coords[0], (test_coords[1])):
            for z in test_coords[2]:
                cells_3D[x, y, z] = 1
    return cells_3D

def convert_to_3D(cell):
    cells_3D = convert_to_3D_numba(cell)
    cells_3D = np.moveaxis(cells_3D, -1, 0)
    cells_3D[cells_3D.shape[0]//2:,:, :] = cells_3D[:cells_3D.shape[0]//2,:, :][::-1]
    return cells_3D
    #cells_3D = np.pad(cells_3D, ((100,100), (50,50), (50,50)))
#cells_3D.shape

In [7]:
@njit
def rounder(x):
    out = np.empty_like(x)
    np.round(x, 0, out)
    return out

In [8]:
additional_width = 0
raster_additional_width = 0

In [9]:
def raster_membrane_cell_3d(raster_cell_length, raster_cell_width, raster_slice_amount):


    membrane_thickness = 0.1 #micron
    raster_membrane_thickness = membrane_thickness/pix_mic_conv * resize_amount
    cell_1 = raster_cell(length=round(raster_cell_length/2)*2, width=round(raster_cell_width/2)*2, additional_width=raster_additional_width)
    cell_2 = raster_cell(length=round((raster_cell_length - raster_membrane_thickness)/2)*2, width=round((raster_cell_width - raster_membrane_thickness)/2)*2, additional_width=raster_additional_width)

    cell_1_3d = convert_to_3D(cell_1)
    cell_2_3d = convert_to_3D(cell_2)

    
    pad_1 = int((cell_1_3d.shape[0] - cell_2_3d.shape[0])/2)
    pad_2 = int((cell_1_3d.shape[1] - cell_2_3d.shape[1])/2)
    pad_3 = int((cell_1_3d.shape[2] - cell_2_3d.shape[2])/2)

    cell_2_3d = np.pad(cell_2_3d, ((pad_1,pad_1), (pad_2, pad_2), (pad_3, pad_3)))
    
    cell_3d = cell_1_3d - cell_2_3d
    
    if raster_slice_amount:
        
        cell_3d = cell_3d[int(raster_slice_amount//2):-int(raster_slice_amount//2),:,:]
    
    return cell_3d

In [10]:
def raster_cell_3d(raster_cell_length, raster_cell_width, raster_slice_amount):

    cell_1 = raster_cell(length=round(raster_cell_length/2)*2, width=round(raster_cell_width/2)*2, additional_width=raster_additional_width)

    cell_1_3d = convert_to_3D(cell_1)
    
    cell_3d = cell_1_3d 
    
    if raster_slice_amount:
        
        cell_3d = cell_3d[int(raster_slice_amount//2):-int(raster_slice_amount//2),:,:]
    
    return cell_3d

In [11]:
wavelengths = [0, 0.4, 0.5, 0.6, 0.7]

def generate_FL_cell(cell_length, cell_width , slice_amount, pad_amount, ID):
    
    raster_cell_length = cell_length/pix_mic_conv * resize_amount
    raster_cell_width = cell_width/pix_mic_conv * resize_amount
    raster_additional_width = additional_width/pix_mic_conv * resize_amount
    raster_slice_amount = slice_amount/pix_mic_conv * resize_amount
    
    
    cell = raster_cell_3d(raster_cell_length,raster_cell_width, raster_slice_amount)
    name_depth, name_length, name_width = cell.shape

    #cell = cell.mean(axis=0)


    FL_cell = np.pad(cell,((0,0),(pad_amount,pad_amount),(pad_amount,pad_amount))).astype(np.float32) 
    
    FL_cells = []
    for wavelength in wavelengths:
        raster_cell_length = cell_length/pix_mic_conv * resize_amount
        raster_cell_width = cell_width/pix_mic_conv * resize_amount
        raster_additional_width = additional_width/pix_mic_conv * resize_amount
        raster_slice_amount = slice_amount/pix_mic_conv * resize_amount
        cell_3d = raster_cell_3d(raster_cell_length,raster_cell_width, raster_slice_amount)
        cell = cell_3d

        name_depth, name_length, name_width = cell_3d.shape
        
        FL_cell = np.pad(cell,((0,0),(pad_amount,pad_amount),(pad_amount,pad_amount))).astype(np.float32) 
        
        if wavelength:
            FL_PSF = PSF_generator(
                radius = radius,
                wavelength = wavelength,
                NA = NA,
                n = n,
                resize_amount = resize_amount,
                pix_mic_conv = pix_mic_conv,
                apo_sigma = apo_sigma,
                mode="3d fluo",
                condenser = "Ph3",
                z_height = int(round(raster_cell_width)),
                pz = 0.5,
                working_distance = 170
            )
            FL_PSF.calculate_PSF()
            #FL_PSF.kernel = np.sum(FL_PSF.kernel, axis=0)
            
        
            #FL_cell_conv = convolve_rescale(image=FL_cell, kernel=FL_PSF.kernel, rescale_factor=1/resize_amount, rescale_int = False)
            PSF_centre = FL_PSF.kernel.shape[0]//2
            cell_centre = FL_cell.shape[0]//2
            cell_conv_idxs = np.arange(FL_cell.shape[0])
            PSF_conv_idxs = np.arange(PSF_centre-cell_centre,PSF_centre+cell_centre)
            FL_cell_conv = np.array(Parallel(n_jobs=1)(delayed(convolve_rescale)(FL_cell[cell_conv_idx].astype(float), FL_PSF.kernel[PSF_conv_idx], 1, False) for cell_conv_idx, PSF_conv_idx in zip(cell_conv_idxs, (PSF_conv_idxs))))
        else:
            
            FL_cell_conv = FL_cell# convolve_rescale(image=FL_cell, kernel=zero_wavelength_PSF, rescale_factor=1/resize_amount, rescale_int = False)
        FL_cell_conv = FL_cell_conv.sum(axis=0)
        np.save(f"single_cells_projection_3d//{ID}_FL_{name_length}_{name_width}_{name_depth}_{raster_additional_width}_{wavelength}.npy", FL_cell_conv)

    
def generate_membrane_cell(cell_length, cell_width, slice_amount, pad_amount, ID, return_array = False):
    
    raster_cell_length = cell_length/pix_mic_conv * resize_amount
    raster_cell_width = cell_width/pix_mic_conv * resize_amount
    raster_slice_amount = slice_amount/pix_mic_conv * resize_amount

    membrane_cell = raster_membrane_cell_3d(raster_cell_length, raster_cell_width, raster_slice_amount)

    name_depth, name_length, name_width = membrane_cell.shape


    membrane_cell = np.pad(membrane_cell,((0,0), (pad_amount, pad_amount), (pad_amount, pad_amount))).astype(np.float32) 
    
    
    
    
    #for wavelength in wavelengths:
    def parallel_render(wavelength):
        
        if wavelength:
            FL_PSF = PSF_generator(
                    radius = radius,
                    wavelength = wavelength,
                    NA = NA,
                    n = n,
                    resize_amount = resize_amount,
                    pix_mic_conv = pix_mic_conv,
                    apo_sigma = apo_sigma,
                    mode="3d fluo",
                    condenser = "Ph3",
                    z_height = int(round(raster_cell_width)),
                    pz = 0.5,
                    working_distance = 170
                )
            FL_PSF.calculate_PSF()
            #FL_PSF.kernel = np.sum(FL_PSF.kernel, axis=0)

            #FL_cell_conv = convolve_rescale(image=FL_cell, kernel=FL_PSF.kernel, rescale_factor=1/resize_amount, rescale_int = False)
            PSF_centre = FL_PSF.kernel.shape[0]//2
            cell_centre = membrane_cell.shape[0]//2
            cell_conv_idxs = np.arange(membrane_cell.shape[0])
            PSF_conv_idxs = np.arange(PSF_centre-cell_centre,PSF_centre+cell_centre)
            membrane_cell_conv = np.array(Parallel(n_jobs=1)(delayed(convolve_rescale)(membrane_cell[cell_conv_idx].astype(float), FL_PSF.kernel[PSF_conv_idx], 1, False) for cell_conv_idx, PSF_conv_idx in zip(cell_conv_idxs, (PSF_conv_idxs))))
 
            
            #membrane_cell_conv = convolve_rescale(image=membrane_cell, kernel=FL_PSF.kernel, rescale_factor=1/resize_amount, rescale_int = False)
        else:
            membrane_cell_conv = membrane_cell # convolve_rescale(image=membrane_cell, kernel=zero_wavelength_PSF, rescale_factor=1/resize_amount, rescale_int = False)
        if return_array:
            return membrane_cell_conv
        membrane_cell_conv = membrane_cell_conv.sum(axis=0)
        np.save(f"single_cells_projection_3d/{ID}_membrane_{name_length}_{name_width}_{name_depth}_{raster_additional_width}_{wavelength}.npy", membrane_cell_conv)
    _ = Parallel(n_jobs=1)(delayed(parallel_render)(wavelength) for wavelength in wavelengths)    
    
def generate_binary_cell(cell_length, cell_width, slice_amount, pad_amount, ID):
    raster_cell_length = cell_length/pix_mic_conv * resize_amount
    raster_cell_width = cell_width/pix_mic_conv * resize_amount
    raster_additional_width = additional_width/pix_mic_conv * resize_amount
    raster_slice_amount = slice_amount/pix_mic_conv * resize_amount
    cell_3d = raster_cell_3d(raster_cell_length,raster_cell_width, raster_slice_amount)
    cell = cell_3d.mean(axis=0)
    binary_image = np.pad(cell, pad_amount) > 0
    binary_image = rescale(binary_image, 1/resize_amount, anti_aliasing=False) > 0
    
    
    raster_depth, raster_cell_length, raster_cell_width = cell_3d.shape
        
    Image.fromarray(img_as_uint(binary_image)).save(f"single_cells_projection_3d/{ID}_binary_{raster_cell_length}_{raster_cell_width}_{raster_depth}_{raster_additional_width}_0.6.png")
    
    #return binary_image

In [12]:
zero_wavelength_PSF = np.array([[0,0,0],[0,1,0],[0,0,0]])
pad_amount = 100

radius = 175
wavelength = 0.65
NA = 1.45
n = 1.518
resize_amount = 1
pix_mic_conv = 0.065 / 6
apo_sigma = 10


In [13]:
cell_length = (3,)
grid_size = 28
max_width = 3.1
cell_width = np.linspace(0.5, max_width, grid_size)
slice_amounts = np.arange(0, 3, np.diff(cell_width)[0]).tolist() 
slice_amounts = slice_amounts + [0]
tolerance = 0.1
param_space = []
widths = []
ID = 0
for length in cell_length:
    for width in cell_width:
        if length >= width:
            for slice_amount in slice_amounts:
                if slice_amount < width:
                    param_space.append([length, width, slice_amount, ID])
                    ID += 1
#param_space = param_space[:16]
print(len(param_space))


507


In [14]:
try:
    os.mkdir("single_cells_projection_3d/")
except:
    pass

In [15]:
8

8

In [ ]:
timeout = 999999999
_ = Parallel(n_jobs=-1, timeout = timeout)(delayed(generate_FL_cell)(length, width, additional_width,  pad_amount, ID) for length, width, additional_width, ID in tqdm(param_space))

  0%|          | 0/507 [00:00<?, ?it/s]

In [81]:
_ = Parallel(n_jobs=-1)(delayed(generate_binary_cell)(length, width, slice_amount,  pad_amount, ID) for length, width, slice_amount, ID in tqdm(param_space))

  0%|          | 0/507 [00:00<?, ?it/s]

/tmp/ipykernel_948525/3654680368.py:123: RuntimeWarning: Mean of empty slice.
/home/gh464/miniconda3/envs/symbac/lib/python3.10/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/tmp/ipykernel_948525/3654680368.py:123: RuntimeWarning: Mean of empty slice.
/home/gh464/miniconda3/envs/symbac/lib/python3.10/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/tmp/ipykernel_948525/3654680368.py:123: RuntimeWarning: Mean of empty slice.
/tmp/ipykernel_948525/3654680368.py:123: RuntimeWarning: Mean of empty slice.
/tmp/ipykernel_948525/3654680368.py:123: RuntimeWarning: Mean of empty slice.
/home/gh464/miniconda3/envs/symbac/lib/python3.10/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/tmp/ipykernel_948525/3654680368.py:123: RuntimeWarning: Mean of empty slice.
/tmp/ipykernel_948525/365468

In [17]:
_ = Parallel(n_jobs=10)(delayed(generate_membrane_cell)(length, width, additional_width,  pad_amount, ID) for length, width, additional_width, ID in tqdm(param_space))

  0%|          | 0/507 [00:00<?, ?it/s]

KeyboardInterrupt: 